# Task 1: Xem xét hành vi khách hàng theo tháng/quý

Notebook này phân tích hành vi mua hàng của khách hàng theo từng giai đoạn thời gian (tháng/quý).

## Mục tiêu

- Phân tích số lượng giao dịch theo tháng/quý
- Phân tích số lượng khách hàng theo tháng/quý
- Phân tích doanh thu theo tháng/quý
- Phân tích số sản phẩm bán được theo tháng/quý
- Phân tích giỏ hàng trung bình theo tháng/quý
- Phân tích hành vi mua hàng theo thời gian


## Parameters


In [ ]:
# PARAMETERS (for papermill)

# Đường dẫn dữ liệu đã làm sạch
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"

# Đơn vị thời gian phân tích: 'month' hoặc 'quarter'
TIME_PERIOD = "month"  # hoặc "quarter"

# Bật/tắt các biểu đồ
PLOT_TRANSACTIONS = True
PLOT_CUSTOMERS = True
PLOT_REVENUE = True
PLOT_PRODUCTS = True
PLOT_BASKET_SIZE = True
PLOT_CUSTOMER_BEHAVIOR = True


## Set up


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from apriori_library import DataVisualizer

# Thiết lập style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

visualizer = DataVisualizer()


## Tải dữ liệu


In [ ]:
# Đọc dữ liệu đã làm sạch
df_clean = pd.read_csv(CLEANED_DATA_PATH, parse_dates=["InvoiceDate"])

print("=== Thông tin dữ liệu ===")
print(f"- Số giao dịch: {df_clean.shape[0]:,}")
print(f"- Khoảng thời gian: {df_clean['InvoiceDate'].min()} đến {df_clean['InvoiceDate'].max()}")
print(f"- Số hoá đơn: {df_clean['InvoiceNo'].nunique():,}")
print(f"- Số khách hàng: {df_clean['CustomerID'].nunique():,}")
print(f"- Số sản phẩm: {df_clean['Description'].nunique():,}")

df_clean.head()


## Chuẩn bị dữ liệu theo thời gian


In [ ]:
# Tạo cột period (tháng hoặc quý)
if TIME_PERIOD == "month":
    df_clean["period"] = df_clean["InvoiceDate"].dt.to_period("M").astype(str)
    period_label = "Tháng"
elif TIME_PERIOD == "quarter":
    df_clean["period"] = df_clean["InvoiceDate"].dt.to_period("Q").astype(str)
    period_label = "Quý"
else:
    raise ValueError("TIME_PERIOD phải là 'month' hoặc 'quarter'")

print(f"=== Phân tích theo {period_label} ===")
print(f"- Số {period_label.lower()} trong dữ liệu: {df_clean['period'].nunique()}")
print(f"- Các {period_label.lower()}: {sorted(df_clean['period'].unique())}")


## Phân tích số lượng giao dịch theo thời gian


In [ ]:
if PLOT_TRANSACTIONS:
    # Số lượng giao dịch theo period
    transactions_by_period = df_clean.groupby("period")["InvoiceNo"].nunique().sort_index()
    
    print("=== Số lượng giao dịch theo thời gian ===")
    display(transactions_by_period.to_frame("Số giao dịch"))
    
    # Vẽ biểu đồ
    plt.figure(figsize=(12, 6))
    transactions_by_period.plot(kind="bar", color="steelblue", edgecolor="black")
    plt.title(f"Số lượng giao dịch theo {period_label}")
    plt.xlabel(period_label)
    plt.ylabel("Số giao dịch")
    plt.xticks(rotation=45, ha="right")
    plt.grid(True, alpha=0.3, axis="y")
    plt.tight_layout()
    plt.show()
    
    # Vẽ đường xu hướng
    plt.figure(figsize=(12, 6))
    transactions_by_period.plot(kind="line", marker="o", linewidth=2, markersize=8, color="green")
    plt.title(f"Xu hướng số lượng giao dịch theo {period_label}")
    plt.xlabel(period_label)
    plt.ylabel("Số giao dịch")
    plt.xticks(rotation=45, ha="right")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


## Phân tích số lượng khách hàng theo thời gian
git status

In [ ]:
if PLOT_CUSTOMERS:
    # Số lượng khách hàng duy nhất theo period
    customers_by_period = df_clean.groupby("period")["CustomerID"].nunique().sort_index()
    
    # Số khách hàng mới (chưa xuất hiện ở period trước)
    new_customers_by_period = {}
    seen_customers = set()
    
    for period in sorted(df_clean["period"].unique()):
        period_customers = set(df_clean[df_clean["period"] == period]["CustomerID"].unique())
        new_customers = period_customers - seen_customers
        new_customers_by_period[period] = len(new_customers)
        seen_customers.update(period_customers)
    
    new_customers_series = pd.Series(new_customers_by_period).sort_index()
    
    print("=== Số lượng khách hàng theo thời gian ===")
    summary_customers = pd.DataFrame({
        "Tổng khách hàng": customers_by_period,
        "Khách hàng mới": new_customers_series
    })
    display(summary_customers)
    
    # Vẽ biểu đồ
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Tổng khách hàng
    axes[0].bar(customers_by_period.index, customers_by_period.values, color="steelblue", edgecolor="black")
    axes[0].set_title(f"Tổng số khách hàng theo {period_label}")
    axes[0].set_xlabel(period_label)
    axes[0].set_ylabel("Số khách hàng")
    axes[0].tick_params(axis="x", rotation=45)
    axes[0].grid(True, alpha=0.3, axis="y")
    
    # Khách hàng mới
    axes[1].bar(new_customers_series.index, new_customers_series.values, color="coral", edgecolor="black")
    axes[1].set_title(f"Số khách hàng mới theo {period_label}")
    axes[1].set_xlabel(period_label)
    axes[1].set_ylabel("Số khách hàng mới")
    axes[1].tick_params(axis="x", rotation=45)
    axes[1].grid(True, alpha=0.3, axis="y")
    
    plt.tight_layout()
    plt.show()


In [ ]:
if PLOT_REVENUE:
    # Doanh thu theo period
    revenue_by_period = df_clean.groupby("period")["TotalPrice"].sum().sort_index()
    
    # Doanh thu trung bình mỗi giao dịch
    avg_revenue_per_transaction = df_clean.groupby("period").agg({
        "TotalPrice": ["sum", "mean"],
        "InvoiceNo": "nunique"
    })
    avg_revenue_per_transaction.columns = ["Tổng doanh thu", "Doanh thu TB/giao dịch", "Số giao dịch"]
    avg_revenue_per_transaction["Doanh thu TB/giao dịch"] = (
        avg_revenue_per_transaction["Tổng doanh thu"] / avg_revenue_per_transaction["Số giao dịch"]
    )
    
    print("=== Doanh thu theo thời gian ===")
    display(avg_revenue_per_transaction)
    
    # Vẽ biểu đồ
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    
    # Tổng doanh thu
    axes[0].bar(revenue_by_period.index, revenue_by_period.values, color="green", edgecolor="black")
    axes[0].set_title(f"Tổng doanh thu theo {period_label}")
    axes[0].set_xlabel(period_label)
    axes[0].set_ylabel("Doanh thu (GBP)")
    axes[0].tick_params(axis="x", rotation=45)
    axes[0].grid(True, alpha=0.3, axis="y")
    
    # Doanh thu trung bình mỗi giao dịch
    axes[1].plot(avg_revenue_per_transaction.index, 
                 avg_revenue_per_transaction["Doanh thu TB/giao dịch"], 
                 marker="o", linewidth=2, markersize=8, color="orange")
    axes[1].set_title(f"Doanh thu trung bình mỗi giao dịch theo {period_label}")
    axes[1].set_xlabel(period_label)
    axes[1].set_ylabel("Doanh thu TB (GBP)")
    axes[1].tick_params(axis="x", rotation=45)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## Phân tích số sản phẩm bán được theo thời gian


In [ ]:
if PLOT_PRODUCTS:
    # Số lượng sản phẩm duy nhất theo period
    products_by_period = df_clean.groupby("period")["Description"].nunique().sort_index()
    
    # Tổng số lượng sản phẩm bán được (quantity)
    quantity_by_period = df_clean.groupby("period")["Quantity"].sum().sort_index()
    
    print("=== Số sản phẩm theo thời gian ===")
    summary_products = pd.DataFrame({
        "Số sản phẩm duy nhất": products_by_period,
        "Tổng số lượng bán": quantity_by_period
    })
    display(summary_products)
    
    # Vẽ biểu đồ
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Số sản phẩm duy nhất
    axes[0].bar(products_by_period.index, products_by_period.values, color="purple", edgecolor="black")
    axes[0].set_title(f"Số sản phẩm duy nhất theo {period_label}")
    axes[0].set_xlabel(period_label)
    axes[0].set_ylabel("Số sản phẩm")
    axes[0].tick_params(axis="x", rotation=45)
    axes[0].grid(True, alpha=0.3, axis="y")
    
    # Tổng số lượng bán
    axes[1].bar(quantity_by_period.index, quantity_by_period.values, color="teal", edgecolor="black")
    axes[1].set_title(f"Tổng số lượng sản phẩm bán được theo {period_label}")
    axes[1].set_xlabel(period_label)
    axes[1].set_ylabel("Số lượng")
    axes[1].tick_params(axis="x", rotation=45)
    axes[1].grid(True, alpha=0.3, axis="y")
    
    plt.tight_layout()
    plt.show()


## Phân tích giỏ hàng trung bình theo thời gian


In [ ]:
if PLOT_BASKET_SIZE:
    # Tính số sản phẩm và giá trị mỗi hóa đơn
    basket_stats = df_clean.groupby(["period", "InvoiceNo"]).agg({
        "Description": "count",  # Số sản phẩm trong giỏ
        "TotalPrice": "sum"      # Giá trị giỏ hàng
    }).reset_index()
    basket_stats.columns = ["period", "InvoiceNo", "ItemsPerBasket", "BasketValue"]
    
    # Thống kê theo period
    basket_by_period = basket_stats.groupby("period").agg({
        "ItemsPerBasket": "mean",
        "BasketValue": "mean"
    }).sort_index()
    
    print("=== Giỏ hàng trung bình theo thời gian ===")
    display(basket_by_period)
    
    # Vẽ biểu đồ
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Số sản phẩm trung bình mỗi giỏ
    axes[0].plot(basket_by_period.index, basket_by_period["ItemsPerBasket"], 
                marker="o", linewidth=2, markersize=8, color="red")
    axes[0].set_title(f"Số sản phẩm trung bình mỗi giỏ hàng theo {period_label}")
    axes[0].set_xlabel(period_label)
    axes[0].set_ylabel("Số sản phẩm")
    axes[0].tick_params(axis="x", rotation=45)
    axes[0].grid(True, alpha=0.3)
    
    # Giá trị trung bình mỗi giỏ
    axes[1].plot(basket_by_period.index, basket_by_period["BasketValue"], 
                marker="s", linewidth=2, markersize=8, color="blue")
    axes[1].set_title(f"Giá trị trung bình mỗi giỏ hàng theo {period_label}")
    axes[1].set_xlabel(period_label)
    axes[1].set_ylabel("Giá trị (GBP)")
    axes[1].tick_params(axis="x", rotation=45)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## Tổng hợp hành vi khách hàng theo thời gian


In [ ]:
if PLOT_CUSTOMER_BEHAVIOR:
    # Tạo bảng tổng hợp
    summary = pd.DataFrame({
        "Số giao dịch": df_clean.groupby("period")["InvoiceNo"].nunique(),
        "Số khách hàng": df_clean.groupby("period")["CustomerID"].nunique(),
        "Tổng doanh thu (GBP)": df_clean.groupby("period")["TotalPrice"].sum(),
        "Số sản phẩm duy nhất": df_clean.groupby("period")["Description"].nunique(),
        "Tổng số lượng bán": df_clean.groupby("period")["Quantity"].sum(),
    }).sort_index()
    
    # Tính thêm các chỉ số
    summary["Doanh thu TB/giao dịch"] = summary["Tổng doanh thu (GBP)"] / summary["Số giao dịch"]
    summary["Giao dịch/khách hàng"] = summary["Số giao dịch"] / summary["Số khách hàng"]
    
    print("=== Tổng hợp hành vi khách hàng theo thời gian ===")
    display(summary.round(2))
    
    # Vẽ biểu đồ tổng hợp
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Số giao dịch
    axes[0, 0].bar(summary.index, summary["Số giao dịch"], color="steelblue", edgecolor="black")
    axes[0, 0].set_title("Số lượng giao dịch")
    axes[0, 0].set_xlabel(period_label)
    axes[0, 0].set_ylabel("Số giao dịch")
    axes[0, 0].tick_params(axis="x", rotation=45)
    axes[0, 0].grid(True, alpha=0.3, axis="y")
    
    # Số khách hàng
    axes[0, 1].bar(summary.index, summary["Số khách hàng"], color="coral", edgecolor="black")
    axes[0, 1].set_title("Số lượng khách hàng")
    axes[0, 1].set_xlabel(period_label)
    axes[0, 1].set_ylabel("Số khách hàng")
    axes[0, 1].tick_params(axis="x", rotation=45)
    axes[0, 1].grid(True, alpha=0.3, axis="y")
    
    # Tổng doanh thu
    axes[1, 0].bar(summary.index, summary["Tổng doanh thu (GBP)"], color="green", edgecolor="black")
    axes[1, 0].set_title("Tổng doanh thu")
    axes[1, 0].set_xlabel(period_label)
    axes[1, 0].set_ylabel("Doanh thu (GBP)")
    axes[1, 0].tick_params(axis="x", rotation=45)
    axes[1, 0].grid(True, alpha=0.3, axis="y")
    
    # Doanh thu trung bình mỗi giao dịch
    axes[1, 1].plot(summary.index, summary["Doanh thu TB/giao dịch"], 
                   marker="o", linewidth=2, markersize=8, color="orange")
    axes[1, 1].set_title("Doanh thu trung bình mỗi giao dịch")
    axes[1, 1].set_xlabel(period_label)
    axes[1, 1].set_ylabel("Doanh thu TB (GBP)")
    axes[1, 1].tick_params(axis="x", rotation=45)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## Kết luận về hành vi khách hàng theo thời gian

Dựa trên phân tích trên, chúng ta có thể rút ra các nhận định sau:

1. **Xu hướng giao dịch**: Xác định được giai đoạn nào có nhiều giao dịch nhất
2. **Tăng trưởng khách hàng**: Phân tích số khách hàng mới và khách hàng quay lại
3. **Doanh thu theo mùa**: Xác định các giai đoạn cao điểm và thấp điểm về doanh thu
4. **Hành vi mua hàng**: Phân tích sự thay đổi về số sản phẩm và giá trị giỏ hàng
5. **Ứng dụng**: Sử dụng thông tin này để tối ưu hóa chiến lược kinh doanh theo thời gian
